In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown as md
import pandas as pd

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [6]:
ticker_excel_book = pd.read_excel('tickers.xlsx', sheet_name=0, index_col=0)

u_after_event_excel_book = pd.read_excel(io='gathered_u_after_event.xlsx', sheet_name=None, index_col=0)

u_event_excel_book = pd.read_excel(io='gathered_u_event.xlsx', sheet_name=None, index_col=0)

leak_excel_book = pd.read_excel(io='gathered_leak.xlsx', sheet_name=None, index_col=0)

vto_excel_book = pd.read_excel(io='gathered_vto.xlsx', sheet_name=None, index_col=0)

mv_excel_book = pd.read_excel(io='gathered_mv.xlsx', sheet_name=None, index_col=0)

ret_excel_book = pd.read_excel(io='gathered_ret.xlsx', sheet_name=None, index_col=0)

vol_excel_book = pd.read_excel(io='gathered_vol.xlsx', sheet_name=None, index_col=0)

cor_df = pd.read_excel(io='gathered_cor.xlsx', sheet_name=0, index_col=0)


In [30]:
case_list = u_after_event_excel_book.copy().keys()
ticker_df = ticker_excel_book.copy()['sector']
cor_series = cor_df.copy().iloc[:,0]
period_first = 1
period_last = 20
period_range = range(period_first, period_last+1)

regression_data_list = []

for case in case_list:
    pro_or_con = case.split('-')[0]
    dpro = int(pro_or_con == 'PRO')
    dcon = int(pro_or_con == 'CON')

    u_0_series = u_event_excel_book[case].copy().iloc[:, 0]
    leak_series = leak_excel_book[case].copy().iloc[:,0]
    ret_series = ret_excel_book[case].copy().iloc[:,0]
    vol_series = vol_excel_book[case].copy().iloc[:,0]
    
    for ticker, sector in ticker_df.items():
        u_0 = u_0_series[ticker]
        leak = leak_series[ticker]
        ret = ret_series[ticker]
        vol = vol_series[ticker]
        cor = cor_series[sector]

        u_after_event_df = u_after_event_excel_book[case]
        vto_df = vto_excel_book[case]
        mv_df = mv_excel_book[case]

        for period in period_range:
            u = u_after_event_df.loc[period, ticker]
            vto = vto_df.loc[period, ticker]
            mv = mv_df.loc[period, ticker]
            regression_data_list.append({
                'CASE': case,
                'PRO_OR_CON': pro_or_con,
                'PERIOD': period,
                'TICKER': ticker,
                'U': u,
                'U_0': u_0,
                'LEAK': leak,
                'VTO': vto,
                'MV': mv,
                'RET': ret,
                'VOL': vol,
                'COR': cor,
                'DPRO': dpro,
                'DCON': dcon,
            })

regression_df = pd.DataFrame(regression_data_list)

In [32]:
with pd.ExcelWriter('regression_data.xlsx') as excel_writer:
    regression_df.to_excel(excel_writer=excel_writer, index=False)